分母：1(房子)+3(大马路)

分子：5(绿色植被) 

In [3]:
import os 
from PIL import Image
from collections import Counter
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import json


%matplotlib notebook

In [4]:
'''
Summary:
    计算两者距离
Parameter：
    xy1 - 坐标1
    xy2 - 坐标2
'''
def cal(xy1, xy2):
    return math.pow((xy1[0] - xy2[0]),2) + math.pow(abs(xy1[1] - xy2[1]),2)

'''
Summary:
    大津阈值分割
Parameter：
    img - 输入图像
'''
def OTSU(img):
    # 类间方差g初始最小
    g_raw = -1
    # 要返回的阈值
    T_return = 0
    # 获得图像大小
    M_N = img.shape[0]*img.shape[1]
    bigest = img.max()
    # 大津阈值算法
    for T in range(bigest):
        # 获取阈值大于T和小于T的两个列表
        array1 = img[img<T]
        array2 = img[img>T]
        # 算出w1和w2
        w1 = len(array1)/M_N
        w2 = len(array2)/M_N
        # 算出μ1和μ2
        if len(array1) == 0:
            mu1 = 0
        else:
            mu1 = sum(array1)/len(array1)
        if len(array2) == 0:
            mu2 = 0
        else:
            mu2 = sum(array2)/len(array2)
        # 算出g
        g=w1*w2*math.pow((mu1-mu2),2)
        if g > g_raw:
            g_raw = g
            T_return = T
    return T_return


'''
Summary:
    绿化程度评分
Parameter：
    average_green - 平均绿视率
'''
def score(average_green):
    if average_green < 0.05:
        return 0
    elif average_green < 0.15:
        return 1
    elif average_green < 0.25:
        return 2
    elif average_green < 0.35:
        return 3
    elif average_green < 0.5:
        return 4
    else:
        return 5

'''
Summary:
    得到绿视率字典
Parameter：
    gree_rate_count - 所有位置的绿视率
Return:
    求平均过后的绿视率
'''
def get_score(gree_rate_count):
    return_dic = {}
    for key, value in gree_rate_count.items():
        #print(sum(value)/len(value))
        average_green = sum(value)/len(value)
        average_green = score(average_green)
        return_dic[key] = average_green
    return return_dic


'''
Summary:
    生成在antv中显示的json文件
Parameter：
    lat_lng_dic - 经纬度+绿化打分
Return:
    json文件的字典
'''
def GetJsonFile(lat_lng_dic):
    DicHead =  {"type":"FeatureCollection","features":[]}
    #DicHead["crs"] = { "type": "name", "properties": { "name": "urn:ogc:def:crs:OGC:1.3:CRS84" } }
    for (lat,lng),value in lat_lng_dic.items():
        lat = float(lat)
        lng = float(lng)
        # 新建每个
        DicEach = {}
        DicEach["type"] = 'Feature'
        DicEach["properties"] = {"mag": value}
        #DicEach["properties"] = {"h0":"4","h1":"2","h2":"1","h3":"0","h4":"0","h5":"0","h6":"1","h7":"1","h8":"4","h9":"4","h10":"4","h11":"4","h12":"4","h13":"2","h14":"3","h15":"4","h16":"3","h17":"3","h18":"2","h19":"2","h20":"4","h21":"5","h22":"5","h23":"6"}

        DicEach["geometry"] = {"type":"Point","coordinates":[lng,lat]}
        DicEach["bbox"] = [lng,lat,lng,lat]
        # 放入头中
        DicHead["features"].append(DicEach)

        
    return DicHead



### 显著区域计算
通过高斯映射得到显著区域

In [5]:
d=np.zeros((320,320))
mid = (160,160)
sigma = 100000
for i in range(320):
    for j in range(320):
        d[i][j] = math.exp(-(cal(mid,(i,j)))/sigma)
# 转换整数
a = d*100
a = a.astype(np.int8) #赋值操作后a的数据类型变化


T = OTSU(a)
b = a
b[b<84] = 0
b[b>=84] = 1
Sp = b


### 绿视率计算
通过输入不同文件名，对绿视率进行计算

In [14]:
filname = 'SanLin'
filpath = './%s/'%filname


gree_rate_count = {}
for pic in os.listdir(filpath):
        if pic[-4:] == '.jpg':
            img = cv2.imread(filpath+pic,0)
            img2 = img.reshape(1,-1)[0]
            count = Counter(img2)
            # 从原始图像提取到马路和建筑的面积
            buildings = count[1]
            road = count[3]
            # 从特征区域提取到的植被的面积
            img2 = img*Sp
            img2 = img2.reshape(1,-1)[0]
            count = Counter(img2)
            tree = count[5]
            # 计算绿视率
            green_rate = tree/(road+buildings)
            # 结果保存
            _,lat,lng,_ = pic.split('_')
            if (lat,lng) not in gree_rate_count:
                gree_rate_count[(lat,lng)] = []
            gree_rate_count[(lat,lng)].append(green_rate)
#     except:
#         print(pic)
lat_lng_dic = get_score(gree_rate_count)
jsonstr = GetJsonFile(lat_lng_dic)
print("平均值",sum(lat_lng_dic.values())/len(lat_lng_dic.values()))
    # 文件写入
filename=filpath+'scores.json'
with open(filname+'.json','w') as file_obj:
    json.dump(jsonstr,file_obj,indent=4)

平均值 3.3363636363636364


#### 各种做实验的空间
不敢删，先留着吧

In [11]:
sum(lat_lng_dic.values())/len(lat_lng_dic.values())

3.8761061946902653

In [63]:
img = cv2.imread(filpath+'_39.952705_116.449084_45.jpg',0)
img2 = img.reshape(1,-1)[0]
count = Counter(img2)
            
img2 = img*Sp
img2 = img2.reshape(1,-1)[0]
count2 = Counter(img2)
            

In [64]:
count

Counter({1: 7722,
         2: 1076,
         0: 3031,
         4: 2207,
         5: 35108,
         3: 52759,
         6: 416,
         7: 37,
         8: 33,
         9: 11})

In [65]:
count2

Counter({0: 49559,
         3: 24981,
         2: 778,
         1: 6746,
         4: 1434,
         5: 18679,
         6: 203,
         7: 10,
         8: 10})

In [28]:
img.reshape(1,-1)

array([[1, 1, 1, ..., 3, 3, 3]], dtype=uint8)

In [29]:
img

array([[1, 1, 1, ..., 6, 5, 5],
       [3, 3, 3, ..., 5, 5, 5],
       [3, 3, 3, ..., 5, 5, 5],
       ...,
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3],
       [3, 3, 3, ..., 3, 3, 3]], dtype=uint8)